In [303]:
from PIL import Image
import cv2
import matplotlib.pyplot as plt
%matplotlib inline
import torch
import numpy as np
import pandas as pd
import pickle as pkl
from torchvision import models, transforms, io
from torch.utils.data import Dataset, DataLoader
import torch.nn.utils.prune as prune
import utils
import os
import time
import copy

# Data Loading

In [18]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
DATASET_PATH = 'ADE20K_2021_17_01/'
index_file = 'index_ade20k.pkl'
with open('{}/{}'.format(DATASET_PATH, index_file), 'rb') as f:
    index_ade20k = pkl.load(f)

objects_mat = index_ade20k['objectPresence']

# Find 150 most common object IDs and non-common object IDs
total_object_counts = np.sum(objects_mat, axis=1)
object_count_ids = np.argsort(total_object_counts)[::-1]
most_common_obj_ids = object_count_ids[:150]
irrelevant_obj_ids = object_count_ids[150:]
# Find image IDs where no irrelevant objects appear
irrelevant_obj_counts = np.sum(objects_mat[irrelevant_obj_ids], axis=0)
good_image_ids = np.argwhere(irrelevant_obj_counts == 0).flatten()
# Only common objects included
common_objects_mat = objects_mat[np.ix_(most_common_obj_ids, good_image_ids)]

# Maps {obj_ids: 0-149}
obj_id_map = {sorted(most_common_obj_ids)[idx]: idx + 1 for idx in range(150)}
obj_id_map[-1] = 0

# Pick out images to train/evaluate on
train_image_ids = []
test_image_ids = []
for i in good_image_ids:
    if 'training' in index_ade20k['folder'][i]:
        train_image_ids.append(i)
    elif 'validation' in index_ade20k['folder'][i]:
        test_image_ids.append(i)
    else:
        raise Exception('Invalid folder name.')

In [5]:
class SegmentationDataset(Dataset):
    def __init__(self, image_ids, root_dir, index_mat, transform=None, target_transform=None):
        """
        Args:
            image_ids (list): list of image IDs from ADE20K
            root_dir (string): Directory with all the images.
            index_mat (array): object array from index_ade20k.pkl
            transform (callable, optional): Optional transform to be applied
                on a sample.
            target_transform (callable, optional): Optional transform to be applied
                on a sample segmentation label.
        """
        self.image_ids = image_ids
        self.root_dir = root_dir
        self.index_ade20k = index_mat
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.image_ids)

    def __getitem__(self, idx):
        image_id = self.image_ids[idx]
        img_name = os.path.join(self.root_dir, self.index_ade20k['folder'][image_id], 
                                self.index_ade20k['filename'][image_id])
        img_info = utils.loadAde20K(img_name)
        
        image = io.read_image(img_info['img_name']).float()
        class_mask = Image.fromarray(img_info['class_mask'], mode='I')
        
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(class_mask)
            
        sample = (image, label)

        return sample

In [6]:
input_size = 224
transform = transforms.Compose([
                transforms.Resize(input_size),
                transforms.CenterCrop(input_size),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            ])

target_transform = transforms.Compose([
                transforms.Resize(input_size, interpolation=0),
                transforms.CenterCrop(input_size),
                transforms.ToTensor()
            ])

/usr/local/lib/python3.7/site-packages/torchvision/transforms/transforms.py:258: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  "Argument interpolation should be of type InterpolationMode instead of int. "


In [35]:
num_samples = 4
batch_size = 2
training_data = SegmentationDataset(train_image_ids[:num_samples], './', index_ade20k, transform=transform, target_transform=target_transform)
train_dataloader = DataLoader(training_data, batch_size=batch_size, shuffle=False)
testing_data = SegmentationDataset(test_image_ids[:num_samples], './', index_ade20k, transform=transform, target_transform=target_transform)
test_dataloader = DataLoader(testing_data, batch_size=batch_size, shuffle=False)

# Load Model

In [298]:
model = models.segmentation.fcn_resnet50(pretrained=False, num_classes=151).to(device=device)

In [299]:
def get_parameter_size(model):
    """
    Return model size in terms of parameters
    Each parameter is a float32 - 4 bytes
    """
    num_params = 0
    for p in model.parameters():
        num_params += torch.count_nonzero(p.flatten())
        
    total_bytes = num_params.item() / 4
    kb = total_bytes / 1000
    
    return kb

In [300]:
print(get_parameter_size(model))

8255.7975


In [301]:
model_copy = copy.deepcopy(model)

matching_flag = True
for p1, p2 in zip(model.parameters(), model_copy.parameters()):
    if p1.data.ne(p2.data).sum() > 0:
        matching_flag = False
print("Copied weights" if matching_flag else "Not matching weights")

Copied weights


In [304]:
for name, module in model_copy.named_modules():
    # prune 20% of connections in all 2D-conv layers
    if isinstance(module, torch.nn.Conv2d):
        p = prune.l1_unstructured(module, name='weight', amount=0.2)
        p = prune.remove(p, name='weight')
    # prune 40% of connections in all linear layers
    elif isinstance(module, torch.nn.Linear):
        prune.l1_unstructured(module, name='weight', amount=0.4)

In [305]:
print(get_parameter_size(model))
print(get_parameter_size(model_copy))

8255.7975
6607.327


In [306]:
for p1, p2 in zip(model.parameters(), model_copy.parameters()):
    if p1.data.ne(p2.data).sum() > 0:
        matching_flag = False
print("Copied weights" if matching_flag else "Not matching weights")

Not matching weights


## Train Model

In [97]:
def encode_label(label_arr, obj_id_map):
    """
    Encode labels for evaluating loss
    label_arr (tensor): B x 1 x H x W
    """
    convert_label_ids = lambda i: obj_id_map[i-1]
    vect_convert_label_ids = np.vectorize(convert_label_ids)
    
    encoded_label = vect_convert_label_ids(label_arr.squeeze().numpy())
    
    return torch.tensor(encoded_label, dtype=torch.long)

In [98]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
criterion = torch.nn.CrossEntropyLoss()
epochs = 2
load_data_start = time.time()
for i in range(epochs):
    print('###### Epoch {} ######'.format(i+1))
    epoch_start = time.time()

    # training pass
    running_loss = 0
    batch_num = 0
    for images, labels in train_dataloader:
        batch_start = time.time()
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        output = model(images)['out']
        labels = encode_label(labels, obj_id_map).to(device)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        batch_num += 1
        print('Batch {}/{} finished... {} seconds'.format(batch_num,len(train_dataloader), time.time() - batch_start))
    print('-----> Training loss: {}'.format(running_loss/len(train_dataloader)))
    print("Training time: {} seconds".format(time.time() - epoch_start))
#     torch.save(model.state_dict(), result_path+'/epochs_{}_weights.pkl'.format(i+1))

    # testing pass
    test_start = time.time()
    running_accuracy = 0
    with torch.no_grad():
        for images, labels in test_dataloader:
            images = images.to(device)
            output = model(images)['out']
            labels = encode_label(labels, obj_id_map).to(device)
            probs = torch.nn.functional.softmax(output, dim=1)
            preds = torch.argmax(probs, dim=1, keepdim=True)
            num_correct = torch.sum((preds == labels).to(int)).item()
            acc = num_correct/(input_size*input_size*len(images))
            running_accuracy += acc
            print('Testing accuracy: {}'.format(num_correct/(input_size*input_size*len(images))))
        print('-----> Overall testing accuracy: {}'.format(running_accuracy / len(test_dataloader)))
    print("Testing time: {} seconds".format(time.time() - test_start))

    print("Epoch completed in {} seconds.".format(time.time() - epoch_start))

print('\n' + '#'*100)
print("DONE TRAINING in {} seconds.".format(time.time() - load_data_start))
print('#'*100 + '\n')

###### Epoch 1 ######
Batch 1/2 finished... 3.423030138015747 seconds
Batch 2/2 finished... 3.5185799598693848 seconds
-----> Training loss: 5.083685398101807
Training time: 7.600463151931763 seconds
Testing accuracy: 0.22341358418367346
Testing accuracy: 0.2294025031887755
-----> Overall testing accuracy: 0.22640804368622447
Testing time: 2.2763829231262207 seconds
Epoch completed in 9.876914978027344 seconds.
###### Epoch 2 ######
Batch 1/2 finished... 3.4821908473968506 seconds
Batch 2/2 finished... 3.726228952407837 seconds
-----> Training loss: 3.1742773056030273
Training time: 7.864411115646362 seconds
Testing accuracy: 0.5626694036989796
Testing accuracy: 0.43714126275510207
-----> Overall testing accuracy: 0.4999053332270408
Testing time: 2.4393069744110107 seconds
Epoch completed in 10.303752183914185 seconds.

####################################################################################################
DONE TRAINING in 20.181429862976074 seconds.
######################

## TRAINING LOOP DOC
Output of running model for each image is (151, 224, 244) - 150 classes + 0 for background, 224 size of image. Currently using batch size 2 for testing, so each generated sample from dataloader is (batchsize, 151, 244, 244)

Label is encoded to be a (244, 244) array of class labels. Classes are adapted to fit top 150 and are mapped ID wise via obj_id_map.